# Data crawling and processing TEST

In [78]:
import urllib.request
import time
import json
import datetime
nextBikeUrl = 'https://api.nextbike.net/maps/nextbike-live.json?city=14'

In [79]:
page = urllib.request.urlopen(nextBikeUrl)
data = page.read()

currentTime = str(datetime.datetime.utcnow())

"""Process the crawled information."""
dataString = data.decode('utf8').replace("'", '"')
dataJson = json.loads(dataString)

In [80]:
bikedata = dataJson["countries"][0]["cities"][0]["places"]

In [81]:
bikedata[1]

{'address': None,
 'bike': False,
 'bike_list': [],
 'bike_numbers': [],
 'bike_racks': 0,
 'bike_types': [],
 'bikes': 0,
 'free_racks': 0,
 'lat': 50.9429485,
 'lng': 6.958015,
 'maintenance': False,
 'name': 'Hauptbahnhof',
 'number': 4826,
 'place_type': '0',
 'rack_locks': False,
 'spot': True,
 'terminal_type': 'free',
 'uid': 35409}

# Data Splitting

In [82]:
bikes = []
stations = []
for i in bikedata:
    if i["bike"] is True:
        bikes.append(i)
    else:
        stations.append(i)

print("Räder", len(bikes))
print("Stationen", len(stations))

Räder 660
Stationen 21


In [83]:
for i in stations:
    print(i["name"])

Bahnhof Mühlheim 
Hauptbahnhof
KVB Hauptverwaltung
Zülpicher Platz
Neumarkt
Uni Köln
Maritim 
Ebertplatz
Hans-Böckler-Platz
Bahnhof Deutz
Breslauer Platz
Severinsstrasse
Wienerplatz
RTL Picassoplatz
Uni Mensa
Chlodwigplatz
TH Köln Südstadt
Leonardo Royal Hotel
Hotel Stadtpalais
Motel One Mediapark
Motel One Waidmarkt


In [84]:
bikes[8]

{'address': '',
 'bike': True,
 'bike_list': [{'active': True,
   'bike_type': 15,
   'boardcomputer': 6555,
   'electric_lock': True,
   'number': '21857',
   'state': 'ok'}],
 'bike_numbers': '21857',
 'bike_racks': 0,
 'bike_types': {'15': 1},
 'bikes': 1,
 'free_racks': 0,
 'lat': 50.9483534,
 'lng': 7.0041173,
 'maintenance': False,
 'name': 'BIKE 21857',
 'number': 0,
 'place_type': '12',
 'rack_locks': False,
 'spot': False,
 'terminal_type': '',
 'uid': 438529}

# Geo Json Format
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
  }
}

In [86]:
def createGeoJsonBikes(bikes, stationid):
    geoJson = {}
    geoJson["geometry"] = { "type": "Point", "coordinates": [bikes["lat"], bikes["lng"]]}
    geoJson["type"] = "Feature"
    geoJson["properties"] = {"address": bikes["address"],
                             "name": bikes["name"],
                             "maintenance": bikes["maintenance"],
                             "state": bikes["bike_list"][0]["state"],
                             "bike_type": bikes["bike_list"][0]["bike_type"],
                             "boardcomputer": bikes["bike_list"][0]["boardcomputer"],
                             "uid": bikes["uid"],
                             "timestamp": currentTime,
                             "stationid": stationid,
                             "stationiduid": stationid,
                             "number": bikes["bike_list"][0]["number"]}
    return geoJson

In [87]:
print(json.dumps(createGeoJsonBikes(bikes[9],0), sort_keys=True, indent=4))

{
    "geometry": {
        "coordinates": [
            50.934139,
            6.88621
        ],
        "type": "Point"
    },
    "properties": {
        "address": "Milit\u00e4rringstra\u00dfe / Junkersdorfer Str. ",
        "bike_type": 0,
        "boardcomputer": 6865,
        "maintenance": false,
        "name": "BIKE 21084",
        "number": "21084",
        "state": "ok",
        "stationid": 0,
        "stationiduid": 0,
        "timestamp": "2017-12-10 15:58:38.667109",
        "uid": 498138
    },
    "type": "Feature"
}


In [88]:
bikelist = []
for i in bikes:
        bikelist.append(createGeoJsonBikes(i, 0))

In [89]:
stations[3]

{'address': None,
 'bike': False,
 'bike_list': [{'active': True,
   'bike_type': 0,
   'boardcomputer': 5217,
   'electric_lock': True,
   'number': '21026',
   'state': 'ok'},
  {'active': True,
   'bike_type': 15,
   'boardcomputer': 7744,
   'electric_lock': True,
   'number': '21549',
   'state': 'ok'},
  {'active': True,
   'bike_type': 15,
   'boardcomputer': 8935,
   'electric_lock': True,
   'number': '21719',
   'state': 'ok'},
  {'active': True,
   'bike_type': 15,
   'boardcomputer': 6369,
   'electric_lock': True,
   'number': '21748',
   'state': 'ok'},
  {'active': True,
   'bike_type': 0,
   'boardcomputer': 5287,
   'electric_lock': True,
   'number': '22457',
   'state': 'ok'}],
 'bike_numbers': '21026,21549,21719,21748,22457',
 'bike_racks': 0,
 'bike_types': {'15': 5, 'undefined': 2},
 'bikes': 5,
 'free_racks': 0,
 'lat': 50.930656880252,
 'lng': 6.9406342506409,
 'maintenance': False,
 'name': 'Zülpicher Platz',
 'number': 4824,
 'place_type': '0',
 'rack_locks': 

In [112]:
def createGeoJsonStations(stations):
    places = []
    bikes = []
    for station in stations:
        placegeojson = {}
        placegeojson["geometry"] = { "type": "Point", "coordinates": [station["lat"], station["lng"]]}
        placegeojson["type"] = "Feature"
        placegeojson["properties"] = {"name": station["name"],
                                 "maintenance": station["maintenance"],
                                 "bike_racks": station["bike_racks"],
                                 "bike_types": station["bike_types"],
                                 "bikes": station["bikes"],
                                 "free_racks": station["free_racks"],
                                 "bike_numbers": station["bike_numbers"],
                                 "rack_locks": station["rack_locks"],
                                 "spot": station["spot"],
                                 "terminal_type": station["terminal_type"],
                                 "timestamp": currentTime,
                                 "number": station["number"],
                                 "uid": station["uid"]}
        
        places.append(placegeojson)
                
                
        for bike in station["bike_list"]:
            geoJson = {}
            geoJson["geometry"] = { "type": "Point", "coordinates": [station["lat"], station["lng"]]}
            geoJson["type"] = "Feature"
            geoJson["properties"] = {"address": station["name"],
                                     "name": station["name"],
                                     "maintenance": "",
                                     "state": bike["state"],
                                     "bike_type": bike["bike_type"],
                                     "boardcomputer": bike["boardcomputer"],
                                     "uid": "",
                                     "timestamp": currentTime,
                                     "stationid": station["number"],
                                     "stationiduid": station["uid"],
                                     "number": bike["number"]}

            bikes.append(geoJson)
    
    return bikes, places

In [113]:
Sbikes,places =  createGeoJsonStations(stations)

In [114]:
len(places)

21

In [103]:
Sbikes

[{'geometry': {'coordinates': [50.938831373138, 6.9062697887421],
   'type': 'Point'},
  'properties': {'address': 'KVB Hauptverwaltung',
   'bike_type': 15,
   'boardcomputer': 6145,
   'maintenance': '',
   'name': 'KVB Hauptverwaltung',
   'number': '21269',
   'state': 'ok',
   'stationid': 4821,
   'stationiduid': 430479,
   'timestamp': '2017-12-10 15:58:38.667109',
   'uid': ''},
  'type': 'Feature'},
 {'geometry': {'coordinates': [50.938831373138, 6.9062697887421],
   'type': 'Point'},
  'properties': {'address': 'KVB Hauptverwaltung',
   'bike_type': 15,
   'boardcomputer': 8834,
   'maintenance': '',
   'name': 'KVB Hauptverwaltung',
   'number': '21908',
   'state': 'ok',
   'stationid': 4821,
   'stationiduid': 430479,
   'timestamp': '2017-12-10 15:58:38.667109',
   'uid': ''},
  'type': 'Feature'},
 {'geometry': {'coordinates': [50.930656880252, 6.9406342506409],
   'type': 'Point'},
  'properties': {'address': 'Zülpicher Platz',
   'bike_type': 0,
   'boardcomputer': 521

In [105]:
for i in Sbikes:
    bikelist.append(i);

In [106]:
len(bikelist)

672

In [107]:
def createCollectionBikes(bikes):
    collection = {"type": "FeatureCollection","features": []}
    for i in bikes:
        collection["features"].append(i)
    return collection

In [115]:
x = createCollectionBikes(bikelist)
y = createCollectionBikes(places)

In [119]:
with open('/Users/christopher/Uni/GDV/Projekt/Gdv-Projekt/data/'
          + 'bikes.geojson', 'w',encoding='utf8') as outfile:
    json.dump(x, outfile ,sort_keys=True, indent=4, ensure_ascii=False)

In [120]:
with open('/Users/christopher/Uni/GDV/Projekt/Gdv-Projekt/data/'
          + 'stations.geojson', 'w',encoding='utf8') as outfile:
    json.dump(y, outfile ,sort_keys=True, indent=4, ensure_ascii=False)